In [1]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, select_volume_data, routes_volume_join, drop_duplicates, export_to_json, qgis_json_format
import pathlib

In [2]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [3]:
#Load raw data
strategic_raw_data, qgis_table = load_data(file, file)

In [4]:
nodes = select_route_data(strategic_raw_data)
ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [5]:
data_sets = {"routes": nodes, "ogv_routes": ogv_nodes}
for user_class, data in data_sets.items():
    nodes_grouped=group_nodes(data)
    links = group_links(nodes_grouped)
    routes = obtain_routes(links, qgis_table)
    volumes=select_volume_data(strategic_raw_data)
    routes = routes_volume_join(routes, volumes)
    unique_routes = drop_duplicates(routes)
    qgis_routes = qgis_json_format(unique_routes)
    export_to_json(user_class, qgis_routes)

Separate gis sheet from same file